Task 1: Multi-layer ANN

Hyperparameters:
 1. Number of nodes
 2. Number of layers
 3. Activation function

In [1]:
# Load library
import pandas as pd
import numpy as np
import math
from enum import Enum
from sklearn.model_selection import train_test_split

In [2]:
# Load data
concrete = pd.read_csv('data/concrete_data.csv')
concrete.head()

,cement,blast_furnace_slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age,concrete_compressive_strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [3]:
# Separate X and Y
# Then separate test and train set
# Also do the Cross-Validation (optional)
X = concrete.drop('concrete_compressive_strength', axis = 1)
y = concrete['concrete_compressive_strength']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.7, random_state = 42)

In [4]:
# Activation function
def logistic(x):
    return 1/(1 + math.exp(-x))

def ReLU(x):
    return max(0, x)

def hyperbolic(x):
    return math.tanh(x)

class ActFunc(Enum):
    log = logistic
    relu = ReLU
    hb = hyperbolic

In [5]:
# Neural Network test!
from neuralNet import neuralNet
from layer import layer

network = neuralNet()

network.add(layer(ActFunc.relu,3))
network.add(layer(ActFunc.hb, 10))
network.add(layer(ActFunc.relu,1))

Applying NN on the concrete data

In [6]:
# PSO test!
import pso

swarmsize = 10
alpha = 0.7
beta = 2
gamma = 1.5
delta = 1.5
epsilon = 0.4
n_iter = 10

particle_swarm_opti = pso.PSO(X_train, y_train, network, swarmsize, alpha, beta, gamma, delta, epsilon, n_iter)
opti_particle, best_mae_arr = particle_swarm_opti.optimise()

Current best mae: 30.00410467208457
Current best mae: 29.52284473300973
Current best mae: 27.98827733046079
Current best mae: 27.89193786119894
Current best mae: 26.275125481798863
Current best mae: 26.009996955930486
Current best mae: 24.768910647272236
Current best mae: 24.67338085258098
Current best mae: 23.471577237922716
Current best mae: 22.797737269789405
Current best mae: 22.14391914486775
Current best mae: 21.98859840767058
Current best mae: 21.48388625324952
Current best mae: 20.738706709608255
Current best mae: 20.471153453900744
Current best mae: 19.685623783828742
Current best mae: 19.264198829674218
Current best mae: 18.886673145442902
Current best mae: 18.320969219413023
Current best mae: 18.06077277853479
Current best mae: 17.869642048028282
Current best mae: 17.579433022105164
Final best mae 17.579433022105164


In [7]:
# Potential ranges of PSO parameters to try
# Could try them like a gridsearch but its probably a bit too much for that, tuning one at a time is probably the way to go

alphaRange = np.arange(0.4,0.9,0.1)
betaRange = np.arange(1.5,2.5,0.1)
gammaRange = np.arange(1.5,2.5,0.1)
deltaRange = np.arange(1.5,2.5,0.1)
epsilonRange = np.arange(0.1,0.5,0.1) # not sure about the range of the learning rate, might need to experiment or look for more sources

In [8]:
#alpha tuning

accuracies = []

for alpha in alphaRange:
    network = neuralNet()

In [ ]:
# Try neural network architectures
# 2-10 layers
# 3-10 neurons per layer

layers = range(1, 5) # 1 to x hidden layer
#neurons = range(2, 30) # 2 to 30 neurons
# Try different layers
# Constant 4 neurons per layer, activation func relu

accuraciesForLayers = []
for numLayers in layers:
    network = neuralNet()
    network.add(layer(ActFunc.hb,4))
    for i in range(numLayers - 1):
        network.add(layer(ActFunc.relu,4))
    network.add(layer(ActFunc.relu, 1)) # output layer
    layerTestPSO = pso.PSO(X_train, y_train, network, swarmsize, alpha, beta, gamma, delta, epsilon, n_iter,prints=False)
    opti_particle, best_mae_arr = layerTestPSO.optimise()
    accuraciesForLayers.append(best_mae_arr[-1])

Current best mae: 33.18710708846312
Current best mae: 32.58017481868509
Current best mae: 32.133144562680656
Current best mae: 31.300867647261608
Current best mae: 30.76489954750443
Current best mae: 30.425243145269768
Current best mae: 30.36212661270656
Current best mae: 29.978573510558036
Current best mae: 29.174864873074256
Current best mae: 29.07512028570915
Current best mae: 28.7120382544829
Current best mae: 27.68508988970189
Current best mae: 27.570786754908095
Current best mae: 27.53204574180525
Current best mae: 27.480306799809174
Current best mae: 27.412671697852517
Current best mae: 26.010339424970198
Current best mae: 25.80237649401427
Current best mae: 24.582267303294394
Current best mae: 24.43276833235693
Current best mae: 23.31403926631566
Current best mae: 23.079620539359404
Current best mae: 22.230529565216134
Current best mae: 22.14854923527954
Current best mae: 22.049729257377887
Current best mae: 21.972991521850492
Current best mae: 21.272900973044113
Current best m

In [33]:
accuraciesForLayers

[20.564607766088397, 12.72089212561704, 12.718980124803672, 12.718731750994262]

In [35]:
neurons = range(3,11)

accuraciesForNeuronCounts = []

for neuronCount in neurons:
    network = neuralNet()
    network.add(layer(ActFunc.hb,neuronCount))
    for i in range(4):
        network.add(layer(ActFunc.relu,neuronCount))
    network.add(layer(ActFunc.relu,1))
    neuronsTestPSO = pso.PSO(X_train, y_train, network, swarmsize, alpha, beta, gamma, delta, epsilon, n_iter,prints=False)
    opti_particle, best_mae_arr = neuronsTestPSO.optimise()
    accuraciesForNeuronCounts.append(best_mae_arr[-1])

In [36]:
accuraciesForNeuronCounts

[12.720326536247986,
 12.719297901296846,
 12.738942461653247,
 12.92201891593844,
 12.749487660926423,
 13.44267954441739,
 13.915180511126492,
 12.149197636654216]

In [ ]:
accuraciesForAlphas = []

for a in alphaRange:
    network = neuralNet()
    network.add(layer(ActFunc.hb,10))
    for i in range(4):
        network.add(layer(ActFunc.relu,10))
    network.add(layer(ActFunc.relu,1))
    alphaTestPSO = pso.PSO(X_train, y_train, network, swarmsize, a, beta, gamma, delta, epsilon, n_iter,prints=False)
    opti_particle, best_mae_arr = alphaTestPSO.optimise()
    accuraciesForAlphas.append(best_mae_arr[-1])

In [ ]:
accuraciesForBetas = []

for b in betaRange:
    network = neuralNet()
    network.add(layer(ActFunc.hb,10))
    for i in range(4):
        network.add(layer(ActFunc.relu,10))
    network.add(layer(ActFunc.relu,1))
    betaTestPSO = pso.PSO(X_train, y_train, network, swarmsize, alpha, b, gamma, delta, epsilon, n_iter,prints=False)
    opti_particle, best_mae_arr = betaTestPSO.optimise()
    accuraciesForBetas.append(best_mae_arr[-1])

KeyboardInterrupt: 

In [ ]:
accuraciesForGammas = []

for g in gammaRange:
    network = neuralNet()
    network.add(layer(ActFunc.hb,10))
    for i in range(4):
        network.add(layer(ActFunc.relu,10))
    network.add(layer(ActFunc.relu,1))
    gammaTestPSO = pso.PSO(X_train, y_train, network, swarmsize, alpha, beta, g, delta, epsilon, n_iter,prints=False)
    opti_particle, best_mae_arr = gammaTestPSO.optimise()
    accuraciesForGammas.append(best_mae_arr[-1])

In [ ]:
accuraciesForDeltas = []

for d in deltaRange:
    network = neuralNet()
    network.add(layer(ActFunc.hb,10))
    for i in range(4):
        network.add(layer(ActFunc.relu,10))
    network.add(layer(ActFunc.relu,1))
    deltaTestPSO = pso.PSO(X_train, y_train, network, swarmsize, alpha, beta, gamma, d, epsilon, n_iter,prints=False)
    opti_particle, best_mae_arr = deltaTestPSO.optimise()
    accuraciesForDeltas.append(best_mae_arr[-1])

In [ ]:
accuraciesForEpsilons = []

for e in epsilonRange:
    network = neuralNet()
    network.add(layer(ActFunc.hb,10))
    for i in range(4):
        network.add(layer(ActFunc.relu,10))
    network.add(layer(ActFunc.relu,1))
    epsilonTestPSO = pso.PSO(X_train, y_train, network, swarmsize, alpha, beta, gamma, delta, e, n_iter,prints=False)
    opti_particle, best_mae_arr = epsilonTestPSO.optimise()
    accuraciesForEpsilons.append(best_mae_arr[-1])

In [ ]:
accuraciesForAlphas

[16.420837013206647,
 12.720444335810935,
 12.718655332856695,
 12.722071306269948,
 12.719791479367064]

In [ ]:
accuraciesForBetas

In [ ]:
accuraciesForGammas

In [ ]:
accuraciesForDeltas

In [ ]:
accuraciesForEpsilons